In [ ]:
imagePath = pkg_resources.resource_filename(‘sparkocr’, ‘resources/ocr/text_detection/020_Yas_patella.jpg’)
binary_data_df = spark.read.format(“binaryFile”).load(imagePath)

In [ ]:
binary_to_image = BinaryToImage()
binary_to_image.setImageType(ImageType.TYPE_3BYTE_BGR)
image_df = binary_to_image.transform(binary_data_df)
display_images(image_df, “image”)

In [ ]:
text_detector = ImageTextDetector.pretrained(“text_detection_v1”, “en”, “clinical/ocr”)
text_detector.setInputCol(“image”)
text_detector.setOutputCol(“text_regions”)
text_detector.setSizeThreshold(10)
text_detector.setScoreThreshold(0.9)
text_detector.setLinkThreshold(0.4)
text_detector.setTextThreshold(0.2)
text_detector.setWidth(1512)
text_detector.setHeight(2016)


In [ ]:
splitter = ImageSplitRegions() \
 .setInputCol(“image”) \
 .setInputRegionsCol(“text_regions”) \
 .setOutputCol(“text_image”) \
 .setDropCols([“image”]) \
 .setExplodeCols([“text_regions”]) \
 .setRotated(True) \
 .setImageType(ImageType.TYPE_BYTE_GRAY)
ocr = ImageToText() \
 .setInputCol(“text_image”) \
 .setOutputCol(“text”) \
 .setPageSegMode(PageSegmentationMode.SINGLE_WORD) \
 .setIgnoreResolution(False)

In [ ]:
documentAssembler = DocumentAssembler() \
 .setInputCol(“text”) \
 .setOutputCol(“document”)
tokenizer = Tokenizer() \
 .setInputCols([“document”]) \
 .setOutputCol(“token”)
entityRuler = EntityRulerApproach() \
 .setInputCols([“document”, “token”]) \
 .setOutputCol(“entities”) \
 .setPatternsResource(
 “patterns.json”,
 ReadAs.TEXT,
 {“format”: “jsonl”}
 ) \
 .setEnablePatternRegex(True)
pipeline_nlp = Pipeline().setStages([
    splitter,
    ocr,
    documentAssembler,
    tokenizer,
    entityRuler
])
text_result = pipeline_nlp.fit(result).transform(result)
text_result.selectExpr("explode(entities)").show(truncate=False)